In [78]:
import folium
from folium.plugins import BeautifyIcon
import pandas as pd
import openrouteservice as ors
# import osmnx


In [79]:
# constants
API_KEY = '5b3ce3597851110001cf6248f7b397efa34d4ff7bc170e361ff89f0b'


In [80]:
client = ors.Client(API_KEY)

# ORS    = [lon, lat]
# folium = [lat, lon]

In [81]:
# get map data of 'LS9 0BT' in osm format.
postcode = "LS9 0BT"
response = client.pelias_search(text=postcode)
response

{'geocoding': {'version': '0.2',
  'attribution': 'https://openrouteservice.org/terms-of-service/#attribution-geocode',
  'query': {'text': 'LS9 0BT',
   'size': 10,
   'private': False,
   'lang': {'name': 'English',
    'iso6391': 'en',
    'iso6393': 'eng',
    'via': 'default',
    'defaulted': True},
   'querySize': 20,
   'parser': 'libpostal',
   'parsed_text': {'postalcode': 'ls9 0bt'}},
  'engine': {'name': 'Pelias', 'author': 'Mapzen', 'version': '1.0'},
  'timestamp': 1712221851388},
 'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-1.517644, 53.789386]},
   'properties': {'id': '470842435',
    'gid': 'whosonfirst:postalcode:470842435',
    'layer': 'postalcode',
    'source': 'whosonfirst',
    'source_id': '470842435',
    'name': 'LS9 0BT',
    'postalcode': 'LS9 0BT',
    'postalcode_gid': 'whosonfirst:postalcode:470842435',
    'confidence': 1,
    'match_type': 'exact',
    'accuracy': 'centroid',
    'co

In [82]:
depot = [53.789386682283094, -1.5176446891452429]

# reverse this before ORS API call

In [83]:
m = folium.Map(location=list(reversed(response['features'][0]['geometry']['coordinates'])), zoom_start=16)
m

In [84]:
# get pothole data (for this region)
pothole_data = pd.read_json(r'./data/one_hole.json')
pothole_data

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'defect': '..."


In [37]:
# posexplode the json to create a df enlisting all defects

# df = pothole_data.explode("features", ignore_index=True)
# df.rename('features'})
# df

,type,feature
0,FeatureCollection,type
1,FeatureCollection,properties
2,FeatureCollection,geometry
3,FeatureCollection,id


In [85]:
# fetch coords of the potholes
coords = []
coords_id = 0

for feature in pothole_data['features']:
    if feature['properties']['defect'] == 'pothole':
        print('pothole: id:', feature['id'], ': dim:', feature['properties']['dim'])
        print ('coordinates:', feature['geometry']['coordinates'])
        coords.append({})
        coords[coords_id]['location'] = feature['geometry']['coordinates']
        coords[coords_id]['service'] = 10*60 # 10min
        coords_id += 1

coords

pothole: id: 0 : dim: 1x0.5x0.2
coordinates: [-1.520265301435614, 53.78860942921949]


[{'location': [-1.520265301435614, 53.78860942921949], 'service': 600}]

In [86]:
# define a vehicle in the fleet
vehicles = [
    ors.optimization.Vehicle(
        id=0, 
        profile='driving-car', 
        start=list(reversed(depot)), 
        # time_window=[0, 5*60*60], 
        end=list(reversed(depot))
        # capacity=[5]
    )
]

vehicles

In [87]:
jobs = [
    ors.optimization.Job(
        id=index, **job
    ) for index, job in enumerate(coords)
]

jobs

In [88]:
# starting from point A (depot) plan route for point B (pothole location).
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)

optimized

{'code': 0,
 'summary': {'cost': 148,
  'routes': 1,
  'unassigned': 0,
  'setup': 0,
  'service': 600,
  'duration': 148,
  'waiting_time': 0,
  'priority': 0,
  'distance': 860,
  'violations': [],
  'computing_times': {'loading': 99, 'solving': 45, 'routing': 23}},
 'unassigned': [],
 'routes': [{'vehicle': 0,
   'cost': 148,
   'setup': 0,
   'service': 600,
   'duration': 148,
   'waiting_time': 0,
   'priority': 0,
   'distance': 860,
   'steps': [{'type': 'start',
     'location': [-1.5176446891452429, 53.789386682283094],
     'setup': 0,
     'service': 0,
     'waiting_time': 0,
     'arrival': 0,
     'duration': 0,
     'violations': [],
     'distance': 0},
    {'type': 'job',
     'location': [-1.520265301435614, 53.78860942921949],
     'id': 0,
     'setup': 0,
     'service': 600,
     'waiting_time': 0,
     'job': 0,
     'arrival': 74,
     'duration': 74,
     'violations': [],
     'distance': 430},
    {'type': 'end',
     'location': [-1.5176446891452429, 53.789

In [89]:
# extract route, decode polyLine geometry to get coordinates of every node/edge.

for route in optimized['routes']:
# has 3 steps:
    # 1. start
    # 2. job
    # 3. end
    geo = route['geometry']
    route_v0 = ors.convert.decode_polyline(route['geometry'])['coordinates']

route_v0

[[-1.5178, 53.7893],
 [-1.51767, 53.78922],
 [-1.51735, 53.78902],
 [-1.51729, 53.78898],
 [-1.51705, 53.78883],
 [-1.51673, 53.78863],
 [-1.51681, 53.78858],
 [-1.51744, 53.78828],
 [-1.51773, 53.78819],
 [-1.51823, 53.78816],
 [-1.51905, 53.78817],
 [-1.52065, 53.7882],
 [-1.5205, 53.78839],
 [-1.52031, 53.7886],
 [-1.52027, 53.7886],
 [-1.52031, 53.7886],
 [-1.5205, 53.78839],
 [-1.52065, 53.7882],
 [-1.51905, 53.78817],
 [-1.51823, 53.78816],
 [-1.51773, 53.78819],
 [-1.51744, 53.78828],
 [-1.51681, 53.78858],
 [-1.51673, 53.78863],
 [-1.51705, 53.78883],
 [-1.51729, 53.78898],
 [-1.51735, 53.78902],
 [-1.51767, 53.78922],
 [-1.5178, 53.7893]]

In [90]:
geo

'cvhgIfmgHNYf@_AFK\\o@f@_AHNz@|BPx@DbBAbDE~He@]i@e@?G?Fh@d@d@\\D_I@cDEcBQy@{@}BIOg@~@]n@GJg@~@OX'

In [91]:
# visualise it in folium map

line_colours = ['orange', 'blue', 'green', 'pink', 'yellow']

for route in optimized['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], weight=2*(3-route['vehicle']), color=line_colours[route['vehicle']]).add_to(m)

m

In [ ]:
# download osm file


In [8]:
# print the route json/xml


In [9]:
# extract the geometry information (polyLine)

In [100]:
# create sumo.trip.xml from point A and point B information
import sumolib

import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")


# Example usage (replace with your actual file paths and coordinates)
net_file = r"osm_sumo_final/sumo_network.net.xml"
start_lat = 53.7893
start_lon = -1.5178
end_lat = 53.7893
end_lon = -1.5178
trip_file = "osm_sumo_final/sumo.trip.xml"

create_trip_file(net_file, start_lat, start_lon, end_lat, end_lon, trip_file)


AttributeError: 'Net' object has no attribute 'getNearestEdgeID'

In [93]:
import subprocess
import os

def generate_sumo_network(osm_file_path, output_directory):
    # Check if SUMO_HOME is set
    if "SUMO_HOME" not in os.environ:
        raise EnvironmentError("SUMO_HOME environment variable not set")

    # Define the path to the netconvert tool
    netconvert_binary = os.path.join(os.environ["SUMO_HOME"], "bin", "netconvert")

    # Define the output file path
    output_file_path = os.path.join(output_directory, "sumo_network.net.xml")

    # Command to convert OSM to SUMO network
    netconvert_command = [
        netconvert_binary,
        "--osm-files", osm_file_path,
        "--output-file", output_file_path,
        "--osm.all-attributes", "false",
        "--plain-output-prefix", os.path.join(output_directory, "osm_net")
    ]

    # Run the command
    subprocess.run(netconvert_command, check=True)
    print(f"SUMO network generated at {output_file_path}")

# Set your OSM file path and desired output directory for the SUMO network files
osm_file_path = r'/Users/shripal/Documents/Study/PathFinding/osm_sumo_final/LS90BT.osm'
output_directory = '.'

generate_sumo_network(osm_file_path, output_directory)


Success.
SUMO network generated at ./sumo_network.net.xml


pj_obj_create: /opt/homebrew/Caskroom/miniforge/base/envs/myml/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 3 is expected. It comes from another PROJ installation.
pj_obj_create: /opt/homebrew/Caskroom/miniforge/base/envs/myml/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 3 is expected. It comes from another PROJ installation.


In [ ]:
### somehow this generated route file is not loading vehicles in the SUMO-GUI


# import urllib

# # Reference for urllib.parse.unquote: https://docs.python.org/3/library/urllib.parse.html
# def convert_to_sumo_route(route_v0):
#   """Converts a decoded polyline into a SUMO route definition.

#   Args:
#       route_v0: A list of tuples containing latitude and longitude coordinates.

#   Returns:
#       A string representing the SUMO route definition in XML format.
#   """
#   coords = urllib.parse.unquote(' '.join(['{},{}'.format(p[0], p[1]) for p in route_v0]))
#   return f'<route id="route1" edges="{coords}">\n  <vehicle id="veh1" type="passenger" depart="0" route="route1"/>\n</route>'

# # Convert the decoded polyline to SUMO route definition
# route_xml = convert_to_sumo_route(route_v0)

# # Write the route definition to sumo.rou.xml file
# with open('sumo.rou.xml', 'w') as f:
#   f.write(route_xml)

# print("Converted polyline and written to sumo.rou.xml")

Converted polyline and written to sumo.rou.xml
